In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [1]:
#Import US DOT insurance history dataset

path = "inshist_allwithhistory.txt"

ins = pd.read_csv(
    path,
    header=None,        
    dtype=str,          
    sep=",",            
    quotechar='"',      
    engine="python",
)
ins.shape

(7135188, 17)

In [3]:
#Change column names (based off of codebook)

ins = ins.rename(columns={
    0: "docket_number",
    1: "usdot_number",
    2: "form_code",
    3: "cancellation_method",
    4: "form_type",
    5: "insurance_type_indicator",
    6: "insurance_type_description",
    7: "policy_number",
    8: "minimum_coverage_amount",
    9: "insurance_class_code",
    10: "effective_date",
    11: "bi_pd_max_underlying_limit_amt",
    12: "bi_pd_max_coverage_amt",
    13: "cancellation_date",
    14: "specific_cancellation_method",
    15: "insurance_company_branch",
    16: "insurance_company_name"
})

In [41]:
# Create transformed data that has one DOT id per row and counts how many times they have changed/cancelled
# insurance, how many unique companies they have used, how many times they've cancelled vs replaced vs transferred,
# the names of the insurance companies they've used, and the types of insurances they've bought

tqdm.pandas()

# Create copy of dataset
ins2 = ins.copy()

# get rid of unusable data (DOT ID 00000000)
ins2["usdot_number"] = ins2["usdot_number"].astype(str).str.strip()
ins2 = ins2[ins2["usdot_number"] != "00000000"]

# Dates
for col in ["effective_date","cancellation_date"]:
    if col in ins2.columns and not pd.api.types.is_datetime64_any_dtype(ins2[col]):
        ins2[col] = pd.to_datetime(ins2[col], errors="coerce")

# Normalize text
ins2["cancellation_method"] = ins2.get("cancellation_method", pd.Series(index=ins2.index, dtype="string"))\
                                 .astype("string").str.strip().str.title()
ins2["form_code"] = ins2.get("form_code", pd.Series(index=ins2.index, dtype="string"))\
                         .astype("string").str.strip().str.upper()

# Features to include in output
tot  = ins2.groupby("usdot_number").size()
fmin = ins2.groupby("usdot_number")["effective_date"].min()
fmax = ins2.groupby("usdot_number")["effective_date"].max()
pc = ins2.groupby(["usdot_number","insurance_company_name"]).size()

num_unique_companies = pc.groupby(level=0).size()
top_company_count    = pc.groupby(level=0).max()
top_company          = pc.groupby(level=0).idxmax().map(lambda k: k[1])
top_company_share    = (top_company_count / tot).fillna(0).astype(float)

all_companies = (
    pc.reset_index()
      .groupby("usdot_number")["insurance_company_name"]
      .apply(lambda s: ", ".join(sorted(pd.unique(s.dropna().astype(str)))))
)

# Keep track of how many unique docket numbers have been used
docket_unique_count = ins2.groupby("usdot_number")["docket_number"].nunique(dropna=True)
all_dockets = (
    ins2.assign(_d=ins2["docket_number"].astype("string"))
        .groupby("usdot_number")["_d"]
        .apply(lambda s: ", ".join(sorted([d for d in pd.unique(s.dropna()) if d != ""])))
)

# Track method that insurance was changed
cancelled_ct  = (ins2["cancellation_method"].eq("Cancelled")).groupby(ins2["usdot_number"]).sum()
replaced_ct   = (ins2["cancellation_method"].eq("Replaced")).groupby(ins2["usdot_number"]).sum()
namechg_ct    = (ins2["cancellation_method"].eq("Name Changed")).groupby(ins2["usdot_number"]).sum()
transfer_ct   = (ins2["cancellation_method"].eq("Transferred")).groupby(ins2["usdot_number"]).sum()

# Track types of insurance that was purchased
def map_form_category(code: str) -> str | None:
    if code is None or code == "" or code is pd.NA:
        return None
    c = str(code).upper()
    if c in {"34", "83"}:
        return "Cargo"
    if c in {"82", "91", "91X"}:
        return "BIPD"
    if c == "84":
        return "BrokerBond"
    if c == "85":
        return "BrokerTrust"
    return None  # ignore other codes for these tallies

form_cat = ins2["form_code"].map(map_form_category)
fc_counts = (
    ins2.assign(_cat=form_cat)
        .dropna(subset=["_cat"])
        .groupby(["usdot_number","_cat"])
        .size()
        .unstack("_cat", fill_value=0)
)

# Ensure all desired columns exist even if zero in this slice
for col in ["Cargo","BIPD","BrokerBond","BrokerTrust"]:
    if col not in fc_counts.columns:
        fc_counts[col] = 0
fc_counts = fc_counts[["Cargo","BIPD","BrokerBond","BrokerTrust"]]
fc_counts = fc_counts.rename(columns={
    "Cargo":"count_cargo",
    "BIPD":"count_bipd",
    "BrokerBond":"count_broker_bond",
    "BrokerTrust":"count_broker_trust_fund"
})

# Assemble final summary (one row per USDOT)
summary = (
    pd.DataFrame({
        "num_filings": tot,
        "num_unique_companies": num_unique_companies,
        "top_company": top_company,
        "top_company_share": top_company_share,
        "cancelled_method_count": cancelled_ct,    
        "replaced_method_count": replaced_ct,      
        "name_changed_method_count": namechg_ct,   
        "transferred_method_count": transfer_ct,  
        "first_filing_date": fmin,
        "last_filing_date": fmax,
        "docket_unique_count": docket_unique_count, 
        "all_dockets": all_dockets,                 
        "all_companies": all_companies,
    })
    .join(fc_counts, how="left")
    .fillna({
        "cancelled_method_count": 0,
        "replaced_method_count": 0,
        "name_changed_method_count": 0,
        "transferred_method_count": 0,
        "count_cargo": 0,
        "count_bipd": 0,
        "count_broker_bond": 0,
        "count_broker_trust_fund": 0,
        "docket_unique_count": 0
    })
    .reset_index()
    .sort_values(["num_unique_companies","num_filings"], ascending=[False, False])
)

# Cast some columns to int
for col in [
    "num_filings","num_unique_companies",
    "cancelled_method_count","replaced_method_count","name_changed_method_count","transferred_method_count",
    "count_cargo","count_bipd","count_broker_bond","count_broker_trust_fund",
    "docket_unique_count"
]:
    summary[col] = summary[col].astype("int64")

summary.head(10)


,usdot_number,num_filings,num_unique_companies,top_company,top_company_share,cancelled_method_count,replaced_method_count,name_changed_method_count,transferred_method_count,first_filing_date,last_filing_date,docket_unique_count,all_dockets,all_companies,count_cargo,count_bipd,count_broker_bond,count_broker_trust_fund
81551,00531158,134,34,CAROLINA CASUALTY INSURANCE CO.,0.10,97,34,3,0,1994-08-19,2024-04-08,1,MC267162,"AMERICAN MOTORISTS INSURANCE CO., AMERICAN SEN...",45,89,0,0
141777,00736291,94,32,CANAL INDEMNITY COMPANY,0.13,71,20,3,0,1999-03-01,2024-09-14,1,MC353574,"CANAL INDEMNITY COMPANY, CANAL INSURANCE CO., ...",19,73,2,0
302982,01403513,49,30,UNDERWRITERS AT LLOYDS LONDON,0.12,28,19,0,2,1992-01-15,2021-04-01,1,MC089617,"ALBANY INSURANCE COMPANY, ASSICURAZIONI GENERA...",21,21,7,0
9637,00107714,84,29,RELIANCE NATIONAL INDEMNITY COMPANY,0.13,30,52,2,0,1994-06-30,2024-07-01,1,MC121119,"ACE AMERICAN INSURANCE COMPANY, AMERICAN EMPIR...",11,70,3,0
47933,00355394,64,29,LINCOLN GENERAL INSURANCE COMPANY,0.09,35,29,0,0,1990-05-11,2023-07-01,1,MC219951,"ACCEPTANCE INDEMNITY INSURANCE CO, ACE FIRE UN...",20,43,1,0
30312,00270179,61,29,LEXINGTON INSURANCE COMPANY,0.13,26,32,3,0,1991-05-01,2024-01-01,1,MC186190,"AMERICAN BONDING COMPANY, AMERICAN CASUALTY CO...",13,28,20,0
73646,00507092,54,29,NEW YORK MARINE & GENERAL INS. CO.,0.09,31,23,0,0,1993-01-15,2020-01-15,1,MC259674,"AMERICAN ALTERNATIVE INSURANCE CORPORATION, AX...",16,31,7,0
27322,00255831,51,29,ZURICH AMERICAN INSURANCE COMPANY,0.10,23,28,0,0,1989-12-27,2020-10-01,1,MC177892,"AEGIS SECURITY INSURANCE COMPANY, ALBANY INSUR...",15,24,12,0
42287,00320691,87,28,SCOTTSDALE INSURANCE COMPANY,0.18,43,41,3,0,1989-04-01,2024-05-01,1,MC210209,"ACE AMERICAN INSURANCE COMPANY, AMERICAN CONTR...",22,41,24,0
73476,00506414,77,28,OCCIDENTAL FIRE AND CASUALTY CO. OF N.C.,0.10,51,25,1,0,1993-12-28,2023-02-03,1,MC259399,"AEQUICAP INSURANCE COMPANY, AMERICAN INTER-FID...",21,55,0,0


In [65]:
# Export to csv
summary.to_csv("insurance_summary.csv", index=False)

In [43]:
census = pd.read_csv("SMS_Input_-_Motor_Carrier_Census_Information_20250919.csv")

   DOT_NUMBER                                   LEGAL_NAME  \
0           1  FEDERAL MOTOR CARRIER SAFETY ADMINISTRATION   
1       10000                   POWELL DISTRIBUTING CO INC   
2     1000000                  JAMES EARL KILLINGSWORTH JR   
3     1000002               NEW JERSEY BOOM & ERECTORS INC   
4     1000004                             RAY TRUCKING LLC   

                       DBA_NAME CARRIER_OPERATION  HM_FLAG  PC_FLAG  \
0     FMCSA TECHNOLOGY DIVISION                 A    False    False   
1                           NaN                 A    False    False   
2  JAMES KILLINGSWORTH TRUCKING                 A    False    False   
3                           NaN                 A    False    False   
4                           NaN                 C    False    False   

                  PHY_STREET      PHY_CITY PHY_STATE PHY_ZIP  ...  \
0  1200 NEW JERSEY AVENUE SE    WASHINGTON        DC   20590  ...   
1             9125 N BURRAGE      PORTLAND        OR   97217  

In [51]:
# insurance dataset: strip leading zeros
ins_ids = summary["usdot_number"].astype(str).str.lstrip("0")

# other dataset: also cast to string (strip spaces just in case)
census_ids = census["DOT_NUMBER"].astype(str).str.strip()

# now compare as sets
a = set(ins_ids)
b = set(census_ids)

matches = a & b
only_ins = a - b
only_other = b - a

output = {
    "match_in_both": len(matches),
    "only_in_insurance": len(only_ins),
    "only_in_other": len(only_other),
    "total_insurance": len(a),
    "total_other": len(b),
}
output


{'match_in_both': 432175,
 'only_in_insurance': 753548,
 'only_in_other': 1659468,
 'total_insurance': 1185723,
 'total_other': 2091643}

In [57]:
summary.shape

(1185723, 18)

In [75]:
# Merge insurance data with census data
def make_dot_key(s):
    return (
        s.astype(str)
         .str.strip()
         .str.lstrip("0")
         .replace("", pd.NA)
    )

census_key  = make_dot_key(census["DOT_NUMBER"])
summary_key = make_dot_key(summary["usdot_number"])

census2  = census.assign(_DOT_KEY=census_key)
summary2 = summary.assign(_DOT_KEY=summary_key)

# Left-join census
merged = census2.merge(
    summary2,
    on="_DOT_KEY",
    how="left",
    suffixes=("_census", "_summary"),
    validate="m:1"
)

# Sanity check that merge occurred correctly
matched_rows   = merged["num_filings"].notna().sum()
unmatched_rows = merged["num_filings"].isna().sum()
total_rows     = len(merged)

matched_rows, unmatched_rows, total_rows


(432175, 1659468, 2091643)

In [79]:
merged

#export to csv

#merged.to_csv("census_with_insurance.csv", index=False)

,DOT_NUMBER,LEGAL_NAME,DBA_NAME,CARRIER_OPERATION,HM_FLAG,PC_FLAG,PHY_STREET,PHY_CITY,PHY_STATE,PHY_ZIP,...,first_filing_date,last_filing_date,docket_unique_count,all_dockets,all_companies,count_cargo,count_bipd,count_broker_bond,count_broker_trust_fund,usdot_number_key
0,1,FEDERAL MOTOR CARRIER SAFETY ADMINISTRATION,FMCSA TECHNOLOGY DIVISION,A,False,False,1200 NEW JERSEY AVENUE SE,WASHINGTON,DC,20590,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000,POWELL DISTRIBUTING CO INC,NaN,A,False,False,9125 N BURRAGE,PORTLAND,OR,97217,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000000,JAMES EARL KILLINGSWORTH JR,JAMES KILLINGSWORTH TRUCKING,A,False,False,15 GASLINE ROAD,PHENIX CITY,AL,36870,...,2020-06-08,2020-06-08,1.00,MC1111926,PROGRESSIVE SPECIALTY INS. CO.,0.00,1.00,0.00,0.00,1000000
3,1000002,NEW JERSEY BOOM & ERECTORS INC,NaN,A,False,False,405 INDUSTRIAL PARK DRIVE,MOUNT POCONO,PA,18344,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000004,RAY TRUCKING LLC,NaN,C,False,False,218 PEACHTREE ST,WARWICK,GA,31796,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2091638,999989,THOMAS C BORCHERT,NaN,C,False,False,W12026 LANGE ROAD,COLUMBUS,WI,53925,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2091639,999990,COPPERHEAD ROAD TRANSPORT LLC,NaN,A,False,False,1069 MINNIE HILL RD,CHILLICOTHE,OH,45601,...,2022-07-29,2022-07-29,1.00,MC1352816,PROGRESSIVE PREFERRED INSURANCE CO.,0.00,1.00,0.00,0.00,999990
2091640,999993,BOB'S DIGGING & LANDSCAPING INC,NaN,C,False,False,W11954 VAN NESS ROAD,LODI,WI,53555,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2091641,999994,BADGER DISPOSAL INC,NaN,C,False,False,150 E SCHOOL STREET,COLUMBUS,WI,53925,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
